In [64]:
import os
import config
%pwd
import mlflow

In [65]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/georgejoshi10/CancerDetection.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'georgejoshi10'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '431f8a5aaf090f3efe9667b7411c28ad8fe32638'

In [66]:
import tensorflow as tf

In [67]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [69]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class EvaluationConfig:
    path_to_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int


In [70]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import *

In [71]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/georgejoshi10/CancerDetection.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [39]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [98]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            batch_size=self.config.params_batch_size,
            image_size=tuple(self.config.params_image_size[:2]),
            labels='inferred',
            label_mode='categorical',
            class_names=['adenocarcinoma', 'normal'],
            subset='validation',
            seed=143,
            color_mode='rgb' if self.config.params_image_size[2] == 3 else 'grayscale',
            validation_split=0.2,
        ).cache().prefetch(tf.data.AUTOTUNE)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            try:
                if tracking_url_type_store != "file":
    
                    # Register the model
                    # There are other ways to use the Model Registry, which depends on the use case,
                    # please refer to the doc for more information:
                    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model")
                    
                logger.info('Model saved Successfully')
            except Exception as e:
                logger.info('Failed saving the model')
                

In [99]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-06-20 13:31:32,788: INFO: common: yaml config/config.yaml loaded successfully]
[2024-06-20 13:31:32,791: INFO: common: yaml params.yaml loaded successfully]
[2024-06-20 13:31:32,792: INFO: common: created directory at: artifacts]
Found 343 files belonging to 2 classes.
Using 68 files for validation.
9/9 [==============================] - 7s 704ms/step - loss: 8.8354e-07 - accuracy: 1.0000
[2024-06-20 13:31:39,665: INFO: common: json file saved at: scores.json]


2024/06/20 13:31:41 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/4n/gbrmm4bs17ldvh4_ypty4z5c0000gn/T/tmp7qx3qd8o/model/data/model/assets
[2024-06-20 13:31:41,757: INFO: builder_impl: Assets written to: /var/folders/4n/gbrmm4bs17ldvh4_ypty4z5c0000gn/T/tmp7qx3qd8o/model/data/model/assets]
[2024-06-20 13:31:41,778: INFO: 3716819959: Failed saving the model]


In [96]:
import keras
keras.__name__



'keras'